In [1]:
%reset -f
!pyclean .

Cleaning directory .
Total 2 files, 1 directories removed.


In [2]:
from utils import *
%matplotlib inline

In [3]:
no_bg_meta = pd.read_csv(os.path.join(DATA_DIRECTORY,"no_background","no_background.csv"),index_col="image_id")
no_bg_meta.head(1)

,spacecraft_id,xmin,ymin,xmax,ymax
image_id,,,,,
23c7f89d00006caee536a22e5807de95,5,994,596,1079,705


In [4]:
no_bg_meta = pd.read_csv(os.path.join(DATA_DIRECTORY,"no_background","no_background.csv"),index_col="image_id").sort_index()
spacecraft_image = [os.path.join(DATA_DIRECTORY,"no_background", "images", i) + ".png" for i in no_bg_meta.index][:40]
spacecraft_bbox = [[k["xmin"], k["ymin"], k["xmax"], k["ymax"]] for i,k in no_bg_meta.iterrows()][:40]
spacecraft_classification = [k["spacecraft_id"]for i,k in no_bg_meta.iterrows()][:40]
spacecraft_mask = [os.path.join(DATA_DIRECTORY,"no_background", "images",i) for i in os.listdir(os.path.join(DATA_DIRECTORY,"no_background", "mask"))]

In [5]:
model = create_mask_fpn_model(input_shape=(224, 224, 720))
model.summary()

Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 7  0           []                               
                                20)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, 224, 224, 16  103696      ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 224, 224, 16  64         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [6]:
dataset = mask_fpn_dataflow(spacecraft_image, spacecraft_mask, batch_size=2)

In [7]:
model.compile(optimizer='adam',
                loss='binary_crossentropy',  # Or 'categorical_crossentropy'
                metrics=['accuracy'])  # Consider adding IoU as a metric

In [8]:
model.fit(dataset, epochs=1)

10/10 [==============================] - 22s 91ms/step - loss: 0.5781 - accuracy: 0.8102
